In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

# GTEx Tissue-Specific RNA-seq Analysis

This notebook template provides a pipeline for the visualization and analysis of RNA-seq gene read counts. Users have the option of either selecting a pre-processed tissue from the [GTEx v8 Data Release](https://gtexportal.org/home/datasets) for analysis, or uploading their own RNA-seq dataset and related metadata. 

### GTEx Data Overview

[The Genotype-Tissue Expression Project](https://gtexportal.org/home/) collects tissue samples from well-genotyped human donors with the aim of better understanding how gene expression and regulation correspond to genetic variation. By analyzing tissue-specific RNA-seq data, it is possible to identify new expression quantitative trait loci (eQTLs) which can provide further insight into the mechanisms through which genetic changes occur in disease states. 

Please refer to the [GTEx Documentation](https://gtexportal.org/home/documentationPage) for specifics on how expression and genotype data were obtained. For the purposes of this analysis, the data was pre-processed as follows:
- non-coding genes were removed by cross-checking with a list of known human genes obtained from the [NCBI Gene Database](https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/)
- for genes with multiple associated Ensembl IDs, only the entry with the highest variance across samples was retained
- the data was divided into tissue-specific sub-datasets of counts from the relevant samples
- metadata for the relevant samples of each tissue-specific dataset has been formatted for readability

Each tissue is associated with two pre-processed datasets:
- *RNA-seq Data*: provides the read counts of each gene (rows) in each sample (columns). 
- *Sample Metadata*: provides metadata for each sample (rows) such as sex, age range, batch IDs, and more (columns). 
The sample_ids found in the metadata table correspond to the column names of the RNA-seq data.

*Note: If you are uploading your own datasets, it is recommended that they are formatted according to the above guidelines. Some functionality may be lost if not all metadata categories are available, or if samples are not correctly labeled by sample_id.*

### Analysis Overview 

The RNA-seq data first undergoes normalization and dimensionality reduction via Principle Component Analysis (PCA) and Uniform Manifold Approximation and Projection (UMAP). Samples are then clustered based on their most-associated highly-variable genes and metadata features. The number of clusters is determined based on a modified silhouette score which prioritizes having more clusters over having larger clusters. Clusters are visualized using the [React-Scatter-Board](https://github.com/MaayanLab/react-scatter-board) package. 

The most up-regulated and down-regulated genes are also identified for each cluster. These genes are used to perform enrichment analysis via the [Enrichr](https://maayanlab.cloud/Enrichr/) API. The enrichment results are visualized with the [React-GSEA](https://github.com/MaayanLab/react-GSEA/tree/simplified) package. 

## 0. Notebook Setup
Import packages and set appropriate file names.

In [ ]:
import os
import numpy as np
import pandas as pd
import requests
import time
from matplotlib import pyplot as plt 
import seaborn as sns
from umap import UMAP
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, silhouette_samples, silhouette_score, plot_roc_curve
from sklearn.cluster import KMeans
import matplotlib.cm as cm
from maayanlab_bioinformatics.dge import characteristic_direction
from maayanlab_bioinformatics.normalization import log2_normalize, filter_by_var, zscore_normalize
from maayanlab_bioinformatics.utils import merge
import math
from collections import OrderedDict
import json
from react_scatter_board.jupyter_compat import ScatterBoard
from IPython.display import display, IFrame, Markdown, HTML
from textwrap import wrap
from react_gsea import ReactGSEA, dataFromResult
from react_gsea.jupyter_compat import ReactGSEA

In [ ]:
# Notebook display util functions
def download_button(content, label, filename):
    # Add download button
    outname = filename.split('.')[0]
    display(HTML('<textarea id="textbox_{outname}" style="display: none;">{content}</textarea> <button style="margin:10px 0;" id="create_{outname}">{label}</button> <a download="{filename}" id="downloadlink_{outname}" style="display: none">Download</a>'.format(**locals())))
    display(HTML('<script type="text/javascript">!function(){{var e=null,t=document.getElementById("create_{outname}"),n=document.getElementById("textbox_{outname}");t.addEventListener("click",function(){{var t,l,c=document.getElementById("downloadlink_{outname}");c.href=(t=n.value,l=new Blob([t],{{type:"text/plain"}}),null!==e&&window.URL.revokeObjectURL(e),e=window.URL.createObjectURL(l)),c.click()}},!1)}}();</script>'.format(**locals())))

def make_clickable(link):
    return f'<a target="_blank" href="{link}">{link}</a>'

def figure_header(label,title):
    display(HTML(f"<div style='font-size:2rem; padding:1rem 0;'><b>{label}</b>: {title}</div>"))
    
def figure_legend(label,title,content=""):
    display(HTML(f"<div style='font-size:1.5rem;'><b>{label}</b>: <i>{title}</i>. {content} </div>"))

In [ ]:
%%appyter hide
{% do SectionField(
    name='TISSUE',
    title='Tissue Selection',
    subtitle='Choose a preprocessed tissue for visualization and analysis. Will run by default if data upload field below is blank.',
    img='gtex_1.png'
)%}

{% do SectionField(
    name='DATASETS',
    title='Dataset Selection',
    subtitle='Upload datasets for visualization and analysis. Both file uploads are required to run the analysis, otherwise the Appyter will default to tissue selection.',
    img='gtex_1.png'
) %}

{% set tissue = ChoiceField(
    name='tissue',
    label='Tissue name',
    description='Select tissue with to analyze.',
    choices={
        'Adipose Tissue': '"AdiposeTissue"',
        'Adrenal Gland': '"AdrenalGland"',
        'Bladder': '"Bladder"',
        'Blood': '"Blood"',
        'Blood Vessel': '"Blood Vessel"',
        'Brain': '"Brain"',
        'Breast': '"Breast"',
        'Cervix': '"CervixUteri"',
        'Colon': '"Colon"',
        'Esophagus': '"Esophagus"',
        'Heart': '"Heart"',
        'Kidney': '"Kidney"',
        'Liver': '"Liver"',
        'Lung': '"Lung"',
        'Muscle': '"Muscle"',
        'Nerve': '"Nerve"',
        'Ovary': '"Ovary"',
        'Pancreas': '"Pancreas"',
        'Pituitary': '"Pituitary"',
        'Prostate': '"Prostate"',
        'Salivary Gland': '"SalivaryGland"',
        'Skin': '"Skin"',
        'Small Intestine': '"SmallIntestine"',
        'Spleen': '"Spleen"',
        'Stomach': '"Stomach"',
        'Testis': '"Testis"',
        'Thyroid': '"Thyroid"',
        'Uterus': '"Uterus"',
        'Vagina': '"Vagina"'
    },
    default='Adrenal Gland',
    section='TISSUE'
) %}

{% set data_filename = FileField(
    name='data_filename',
    label='RNA-seq data file',
    description='TSV or CSV file containing RNA-seq read counts. Index should be Entrez gene symbols, and columns should be individual samples.',
    default='',
    examples={
        'Brain Tissue': 'https://appyters.maayanlab.cloud/storage/GTEx_Tissue_RNAseq_Analysis/Brain_counts_deduped.tsv'
    },
    section='DATASETS'
) %}

{% set metadata_filename = FileField(
    name='metadata_filename',
    label='Sample metadata file',
    description='CSV or TSV file containing sample metadata. Index should be sample IDs corresponding to columns of RNA-seq data file, and columns should be different sample attributes.',
    default='',
    examples={
        'Brain Tissue Metadata': 'https://appyters.maayanlab.cloud/storage/GTEx_Tissue_RNAseq_Analysis/metadata/metadata_Brain_counts_deduped.tsv'
    },
    section='DATASETS'
) %}

In [ ]:
%%appyter code_exec

tissue = {{ tissue }}
data_filename = {{ data_filename }}
metadata_filename = {{ metadata_filename }}

if (data_filename == '' or metadata_filename == ''):
    data_filename = 'https://appyters.maayanlab.cloud/storage/GTEx_Tissue_RNAseq_Analysis/' + tissue + '_counts_deduped.tsv'
    metadata_filename = 'https://appyters.maayanlab.cloud/storage/GTEx_Tissue_RNAseq_Analysis/metadata/metadata_' + tissue + '_counts_deduped.tsv'

## 1. Import Datasets
Load RNA-seq gene read counts and associated sample metadata into dataframes.

In [ ]:
def load_dataframe(file):
    ''' Load a file by downloading it or reading it if already downloaded.
    '''
    ext = os.path.splitext(file)[1]
    if ext in {'.tsv', '.txt'}:
        df = pd.read_csv(file, sep='\t', index_col=0)
    elif ext == '.csv':
        df = pd.read_csv(file, index_col=0)
    else:
        raise Exception('Unrecognized file format', ext)

    # Fix any type coersion on identifiers
    df.index = df.index.astype(str)
    df.columns = df.columns.astype(str)

    return df

In [ ]:
data_index = "symbol"
metadata_index = "sample_id"

print(f"Loading user-uploaded data...")
df_data = load_dataframe(data_filename).sort_index()
df_metadata = load_dataframe(metadata_filename).sort_index()

df_data.index.name = "symbol"
df_metadata.index.name = "sample_id" 

print("Data loaded!")

### 1a. RNA-seq Data

In [ ]:
figure_legend("Table 1", "RNA-seq data", "The RNA-seq data contains a row per gene and a column per sample. The column indices are unique GTEx sample_ids, while the row indices are Entrez gene symbols.")
display(df_data.head())

### 1b. Metadata

#### Phenotype Metadata
Three categories of phenotypic attributes are collected and published for each donor on the GTEx portal.
- Sex 
    - **1**=Male
    - **2**=Female
- Age
- Hardy Scale Death Classification 
    - **0**=Ventilator case
    - **1**=Fast and violent death
    - **2**=Fast death from natural causes
    - **3**=Intermediate death
    - **4**=Slow death
    
During preprocessing, the sex and Hardy Scale classifications were re-coded as strings, to increase readability. 

#### Sample Metadata
There are also 62 different sample attributes recorded for each sample, which are described more in-depth [in this Excel spreadsheet](https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDD.xlsx). 

#### Combined Metadata
In order to gain a more holistic understanding of each sample and the individual from whom it was obtained, the sample and donor metadata were combined during preprocessing by mapping each sample ID from the sample metadata to the matching donor ID in the phenotype metadata, and then prepending the associated phenotype values to the sample metadata dataframe.

Since not all categories have unique values across all samples, any metadata categories with less than 2 unique values across all samples are removed, with the exception of the sex category due to sex-specific tissues. On the other hand, categories which have a unique value for every sample aren't very helpful either, so those are removed as well. 

In [ ]:
drop_cols = []

for (col, items) in df_metadata.iteritems():
    # attribute columns besides sex with at least two unique values
    uniq_vals = len(items.unique()) > 1 or col == 'sex' 
    # attribute columns with at least one repeated value among samples
    less_than_total = len(items.unique()) < df_metadata.shape[0]
    # add columns not fitting above criteria to list of columns to be dropped
    if not (uniq_vals and less_than_total):
        drop_cols.append(col)

df_metadata = df_metadata.drop(columns=drop_cols)

In [ ]:
figure_legend("Table 2","Combined metadata", "The column indices refer to relevant sample and donor metadata categories, while the row indices are unique GTEx sample IDs corresponding to the columns of the RNA-seq data table.")
display(df_metadata.head())

Listed below are all the metadata categories with at least two unique values across the samples, but less than the total number of samples. These will be relevant during the dimensionality reduction and clustering steps.

In [ ]:
features = df_metadata.columns.values
print(features)

## 2. Normalize Data
Given the highly variable nature of expression level between different genes, it is necessary to normalize the read counts before proceeding.

In [ ]:
# create dataframe to display sample stats
df_library_size = pd.DataFrame(
    {
        'n_reads': df_data[df_data > 0].count(),
        'log_n_reads': np.log2(df_data[df_data > 0].count() + 1),
        'n_expressed_genes': df_data.sum(),
    }).sort_values('n_reads', ascending=False)

df_library_size.index.name = "sample_id"

In [ ]:
figure_legend("Table 3","Library size", "By default, the first five entries are shown. A gene read is counted toward n_reads for a single sample if its value is greater than 0.")
display(df_library_size.head()) 

In [ ]:
figure_legend("Figure 1","Library size distribution")
sns.distplot(df_library_size["n_reads"]); plt.show()

Two versions of the dataset are normalized: one with just the top 2500 most variable genes and one with all genes. The former will be used to compute clusters after dimensionality reduction, and the latter to compute the characteristic direction (up or down) of each gene in a cluster. 

In [ ]:
# copy full dataset for computing characteristic directions later
df_data_norm_all_genes = df_data.copy()

# take top 2500 most variable rows
df_data_norm = filter_by_var(df_data,top_n = 2500)

# compute log normalization of matrix
df_data_norm = log2_normalize(df_data_norm)
df_data_norm_all_genes = log2_normalize(df_data_norm_all_genes)

# convert to zscores
df_data_norm = zscore_normalize(df_data_norm)
df_data_norm_all_genes = zscore_normalize(df_data_norm_all_genes)

In [ ]:
figure_legend("Table 4","Normalized RNA-seq data for 2500 most variable genes", "Counts are filtered for the top 2500 most variable genes. The resulting dataset is log transformed and normalized, then converted to z-scores.")
display(df_data_norm.head())

In [ ]:
# plot the first gene distribution
gene = df_data_norm.index.values[0]
sns.distplot(df_data_norm.iloc[0, :]); plt.show()
figure_legend("Figure 2",f"Sample gene expression distibution for {gene}", f"In this dataset, {gene} is the most variably expressed across all samples.")

# plot the last gene distribution
gene = df_data_norm.index.values[-1]
sns.distplot(df_data_norm.iloc[-1, :]); plt.show()
figure_legend("Figure 3",f"Sample gene expression distibution for {gene}", f"In this dataset, {gene} is the least variably expressed across all samples among the 2500 most variably expressed genes.")

# plot a single RNA-seq sample distribution
sns.distplot(df_data_norm.iloc[:, 0]); plt.show()
figure_legend("Figure 4","Single GTEx sample RNA-seq profile distribution")

## 3. Reduce Data Dimensionality

Now that the data has been loaded and normalized, the most variable genes across the dataset can be identified and visualized with hierachical clustering and heatmaps. Dimensionality reduction facilitates the differentiation of the data in a more efficient manner by reducing the number of attributes to be considered. 

### 3a. Principle Component Analysis 
PCA is used first to reduce the dimensionality of the dataset, while still maintaining most of the variability. In PCA, a large number of dimensions -- in this case, the different sample metadata attributes -- can be reduced to a few new dimensions that capture the relevant information of the original attributes. This analysis uses 10 PCA components.

In [ ]:
# PCA
data_norm_pca = PCA(
  random_state=42,
)

data_norm_pca.fit(df_data_norm.values.T)

df_data_norm_pca = pd.DataFrame(
    data_norm_pca.transform(df_data_norm.values.T),
    index=df_data_norm.T.index
)

df_data_norm_pca.columns = [
    f'PCA-{c}' # ({r:.3f})'
    for c, r in zip(df_data_norm_pca.columns, data_norm_pca.explained_variance_ratio_)
]

df_data_norm_pca.index.name = "sample_id"

In [ ]:
figure_legend("Table 5","Principle components of RNA-seq data", "The top principle components are the projections of each datapoint onto the axes along which there is the most variation in the dataset.")
display(df_data_norm_pca.head())

The data can now be plotted with the [React-Scatter-Board](https://github.com/MaayanLab/react-scatter-board) package. The points can be shaped and colored by various metadata categories, with the default being age and sex. They can also be individually searched by sample_id.

In [ ]:
# combine metadata with RNA-seq data; note this will fail if sample_ids are
# not exactly matched between both datasets
pca_data = merge(
        df_data_norm_pca[["PCA-0", "PCA-1"]],
        df_library_size,
        df_metadata
      )

# name columns for plotting purposes
pca_data = pca_data.rename(columns={'PCA-0': 'x', 'PCA-1': 'y'})
pca_data['sample_id'] = pca_data.index

# normalize dimensions to -10, 10
pca_min, pca_max = -10, 10

pca_x_min, pca_x_max = pca_data['x'].min(), pca_data['x'].max()
pca_y_min, pca_y_max = pca_data['y'].min(), pca_data['y'].max()
pca_data['x'] = (pca_data['x'] - pca_x_min) / (pca_x_max - pca_x_min) * (pca_max - pca_min) + pca_min
pca_data['y'] = (pca_data['y'] - pca_y_min) / (pca_y_max - pca_y_min) * (pca_max - pca_min) + pca_min

In [ ]:
pca_scatter_data = pca_data.to_dict('records')

# for the GTEx data, points are shaped by age and colored by sex
# for user-uploaded data, shape/color will reflect your column order
shape_def = df_metadata.columns.values[1]
color_def = df_metadata.columns.values[0]

ScatterBoard(
    id='pca-scatterboard',
    is3d=False,
    data=pca_scatter_data,
    shapeKey=shape_def,
    colorKey=color_def,
    labelKeys=['sample_id'],
    searchKeys=['sample_id'],
    width=700,
    height=400
)


**Figure 5:** *First two PCA components of RNA-seq data.* Points are labeled by Sample ID and can be color- or shape-coded by any of the metadata categories using the dropdown menus. Points can also be isolated by searching by sample ID. Scroll to zoom, drag to move around.

### 4b. Uniform Manifold Approximation and Projection

The dimensionality of the dataset is further reduced by performing UMAP on the 10 PCA components. Parameters such as `n_neighbors` and `min_dist` are set according to defaults used by the Seurat R Package for single cell genomics analysis.

In [ ]:
data_norm_umap = UMAP(
  random_state=42,
  n_components=2,
  n_neighbors=min(40, df_data_norm_pca.shape[0]-1),
  metric='cosine',
  min_dist=0.3,
)

# use top 10 components of PCA
n_pca_components = min(10,df_data_norm_pca.shape[1])
data_norm_umap.fit(df_data_norm_pca.iloc[:, :n_pca_components].values)

# keep only first two UMAP components
df_data_norm_umap = pd.DataFrame(
  data_norm_umap.transform(df_data_norm_pca.iloc[:, :n_pca_components].values),
  columns=['UMAP-0', 'UMAP-1'],
  index=df_data_norm_pca.index,
)

In [ ]:
# project data onto first two UMAP components for visualization
umap_data = merge(
        df_data_norm_umap[["UMAP-0", "UMAP-1"]],
        df_library_size,
        df_metadata
      )

umap_data = umap_data.rename(columns={'UMAP-0': 'x', 'UMAP-1': 'y'})
umap_data['sample_id'] = umap_data.index

umap_min, umap_max = -10, 10

umap_x_min, umap_x_max = umap_data['x'].min(), umap_data['x'].max()
umap_y_min, umap_y_max = umap_data['y'].min(), umap_data['y'].max()
umap_data['x'] = (umap_data['x'] - umap_x_min) / (umap_x_max - umap_x_min) * (umap_max - umap_min) + umap_min
umap_data['y'] = (umap_data['y'] - umap_y_min) / (umap_y_max - umap_y_min) * (umap_max - umap_min) + umap_min

In [ ]:
umap_scatter_data = umap_data.to_dict('records')
shape_def = df_metadata.columns.values[1]
color_def = df_metadata.columns.values[0]

ScatterBoard(
    id='umap-scatterboard',
    is3d=False,
    data=umap_scatter_data,
    shapeKey=shape_def,
    colorKey=color_def,
    labelKeys=['sample_id'],
    searchKeys=['sample_id'],
    width=700,
    height=400
)

**Figure 6:** *First two UMAP components of RNA-seq data.* The datapoints are again labeled by sample ID, and can be color- or shape-coded by any of the metadata categories using the dropdown menu. Points can also be isolated by searching by sample ID. Scroll to zoom, drag to move around.

## 5. Clustering

The first two UMAP components will be used from here on out. 

To compute sample clusters, the k-means method is used. The total number of clusters must be determined, by first testing a range for the number of total clusters, and then computing silhouette scores, which are a measure of how similar an entry is to its own cluster versus other clusters. The goal is to maximize both the similarity within a cluster and the differences between clusters, so the ideal number of clusters is that which produces the highest silhouette score.

Here, the selection of $k$ is modified slightly to preserve nuances in the data by prioritizing more clusters over fewer. A modified silhouette score is also calculated, which takes into account the number of clusters $k$; each original score is bumped by a factor linear with respect to $k$. 

In [ ]:
silhouette_scores = []

# Set max clusters as a function of the sample size and the user-selected option
max_clusters = math.ceil(df_data_norm_umap.shape[0]**0.5)
max_clusters = int(math.ceil(max_clusters/2))

# function for weighting results with more clusters
def calc_weighted_score(sil_score, k, max_k):
    return sil_score*0.7 + k/max_k*0.3

cluster_range = range(2, (max(max_clusters, 3)))
for n in cluster_range:
    # apply k-means clustering for each possible k
    X = df_data_norm_umap.values
    clusterer = KMeans(n_clusters=n, random_state=42).fit(X)
    y_pred = clusterer.predict(X)
    
    # The silhouette_score gives the average value for all the samples
    silhouette_avg = silhouette_score(X, y_pred, metric='cosine')
    
    # Compute a weighted score that rewards higher numbers of clusters
    weighted_score = calc_weighted_score(silhouette_avg, n, max_clusters)

    silhouette_scores.append({
        "N Clusters": n,
        "Silhouette Score": silhouette_avg,
        "Weighted Score": weighted_score
    })
    
    # Labeling the clusters
    centers = clusterer.cluster_centers_

In [ ]:
# use weighted scores for 
points = {}
threshold = 0.3
    
for s in silhouette_scores:
    points[s["N Clusters"]] = s["Weighted Score"]

# No local maxima - resort to previous method of best score calculation
k = None

In [ ]:
silhouette_scores = pd.DataFrame(silhouette_scores)

figure_legend("Table 6", "Silhouette scores by number of clusters", "Values are sorted by the highest silhouette score.")
display(silhouette_scores.sort_values(["Silhouette Score", "Weighted Score"], ascending=False).reset_index().head())

best_score = silhouette_scores.sort_values('Silhouette Score').iloc[-1].to_dict()
best_weighted = silhouette_scores.sort_values('Weighted Score').iloc[-1].to_dict()

best = {"Silhouette Score": best_score, "Weighted Score": best_weighted}

In [ ]:
if not k:
    k = int(best["Weighted Score"]["N Clusters"])
    
print(f"Ideal k: {k} clusters")

In [ ]:
# plot both the uneqeighted and weighted scores as a function of # of clusters
colors = {"Silhouette Score": "#7C88FB", "Weighted Score": "#00CC96"}

for score_type in ['Silhouette Score', 'Weighted Score']:
    plt.plot(silhouette_scores['N Clusters'], silhouette_scores[score_type], label=score_type,color=colors[score_type])
    plt.scatter([best[score_type]['N Clusters']], [best[score_type][score_type]], label=f"Best {score_type}: {int(best[score_type]['N Clusters'])} clusters",color=colors[score_type])
plt.axvline(k, label = f"Ideal k: {k} clusters", color ="#EF553B", alpha=0.8,dashes=(3,3))
plt.legend()
plt.ylabel('Score')
plt.xlabel('Number of Clusters')
plt.show()
figure_legend("Figure 7", "Cluster size selection", "The dotted line indicates the value of the 'ideal' <i>k</i> as chosen by the selected scoring method. This value will be used in subsequent clustering.")

In [ ]:
# Compute the k-means dataframe using the ideal number of clusters
km = KMeans(n_clusters=k, random_state=42)
km_clusters = km.fit_predict(df_data_norm_umap.values)

df_data_norm_km = pd.DataFrame({
'Cluster': [
    str(c)
    for c in km_clusters
]}, index=df_data_norm_umap.index)


print(f'Computed {len(df_data_norm_km["Cluster"].unique())} clusters')

In [ ]:
# Map each cluster to a color for later plots
clusters = df_data_norm_km["Cluster"].unique()
plotly_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
cluster_colors = {}
i = 0
for c in clusters:
    cluster_colors[c] = plotly_colors[i % len(plotly_colors)]
    i += 1

def cluster_heading(cluster):
    display(HTML(f'''
    <center>
    <div style='background-color:{cluster_colors[cluster] + '98'};
        width:100%;height:3rem;display:flex;align-items:center;
        justify-content:center;color:white;font-size:2rem'>
        <center>Cluster {cluster}</center>
    </div>
    </center>'''))
    

Next, the differential expression for each cluster is computed. The <a href="http://www.maayanlab.net/CD/">Characteristic Direction method</a> is used for identifying differentially expressed genes among the different clusters.

In [ ]:
# Get differential expression for each cluster, using the dataset containing all genes
diff_expr = {}
for cluster, samples in df_data_norm_km.groupby('Cluster'):
    diff_expr[f"Cluster {cluster} CD"] = characteristic_direction(
        # expression outside of this cluster
        df_data_norm_all_genes.loc[:, df_data_norm_all_genes.columns.difference(samples.index)],
        # expression in this cluster
        df_data_norm_all_genes.loc[:, samples.index],
      )['CD-coefficient']

df_diff_expr = pd.DataFrame(diff_expr)
df_diff_expr = df_diff_expr.sort_values(by='Cluster 0 CD',ascending=True)
df_diff_expr['Symbol'] = df_diff_expr.index.values

In [ ]:
figure_legend("Table 7", "Differential expression of genes by cluster", "By default, the top 5 most differentially expressed genes are shown, along with the corresponding characteristic directions for each cluster.")
display(df_diff_expr.head())

Logistic regression is performed for each metadata category to determine which categories most accurately predict cluster designations for each data point. ROC curves are also plotted for categories with the top two highest AUC scores.

In [ ]:
# LR
aucs = {}
rocs = {}

for cluster, samples in df_data_norm_km.groupby('Cluster'): 
    aucs[cluster] = {}
    rocs[cluster] = []

    for feature in features:
        lr = LogisticRegression()
        X = df_metadata.copy()
        X = X[feature]
        X = pd.merge(X, df_data_norm_km, left_index = True, right_index = True)

        # drop NAs, and move on if dataset is empty
        X.replace("not reported", None)
        X = X.dropna()
        if (X.shape[0] == 0): continue

        cluster_data = X["Cluster"]
        X = X.drop(columns= ["Cluster"])

        # one-hot encode non numerical data
        if (not isinstance(X[feature][0], (int, float, complex))):
            X = pd.get_dummies(X[feature], prefix=feature)

        y_true = (cluster_data == cluster)
        
        if (len(y_true.unique()) < 2): # if there is only one class in the dataset
            print(f"Not enough data to classify cluster {cluster} based on category {feature}")
            aucs[cluster][feature] = np.nan
            continue 
                  
        lr.fit(X, y_true)

        y_score = lr.predict_proba(X)[:, 1]
        auc_score = roc_auc_score(y_true, y_score)
        aucs[cluster][feature] = auc_score
        
        # save the ROCs
       
        rocs[cluster].append({"auc":auc_score, "lr": lr, "X": X, "y_true":y_true, "title": f'Predictions of cluster {cluster} by category {feature}'})
        
df_cluster_aucs = pd.DataFrame(aucs)
df_cluster_aucs.index.name="Category"

# sort features by avg AUC across all clusters
df_cluster_aucs["avg"] = [ np.mean(df_cluster_aucs.T[f]) for f in df_cluster_aucs.index.values ]
df_cluster_aucs = df_cluster_aucs.sort_values(by = "avg", ascending=False)
df_cluster_aucs = df_cluster_aucs.drop(columns = "avg")

cols = [('Cluster', col) for col in df_cluster_aucs.columns ]
df_cluster_aucs.columns = pd.MultiIndex.from_tuples(cols)

In [ ]:
figure_legend("Table 8", "Average AUC scores for top predictive metadata categories, by cluster", "Scores for the top 5 metadata categories for predicting clusters, as determined by the average AUC score across all clusters, are shown. Higher AUC scores correspond to better classifiers for distinguishing whether or not a datapoint belongs to a certain cluster.")
display(df_cluster_aucs.head(5))

In [ ]:
# plot top 2 ROCs for each cluster
plt.rc('font', size=16)

for cluster, plots in rocs.items():
    plots.sort(reverse=True, key=lambda x: x["auc"])
    cluster_heading(cluster)
    
    if len(plots) < 2:
        best_rocs = plots
    else:
        best_rocs = plots[:2]

    num_plots = len(best_rocs)
    figure,axes = plt.subplots(int(math.ceil(num_plots / 2.)), 2, figsize=(15,(len(best_rocs)*3.5)))
    
    axes = axes.flatten()
    for i in range(len(axes)):
        if i >= len(best_rocs):
            axes[i].remove()
        else:
            plot = best_rocs[i]
            fig = plot_roc_curve(plot["lr"], plot["X"], plot["y_true"], ax=axes[i])

            axes[i].set_title('\n'.join(wrap(plot["title"], 40)))

    figure.tight_layout(pad=2)
    plt.show()
    
figure_legend("Figure 8", "ROCs for top cluster-predicting metadata categories")

plt.rcdefaults()

## 6. Identify Up- and Down-Regulated Genes
Find the most up- and down-regulated genes for each cluster for visualization in heatmap, and for enrichment analysis. 

In [ ]:
# Merge data
df_clustered_umap = pd.merge(left=df_data_norm_km, left_on="sample_id", right=df_data_norm_umap, right_on="sample_id")

In [ ]:
# Get top Genes for each cluster
top_genes = {}
all_top_genes = []
heatmap_top_n = 50
for cluster in df_clustered_umap['Cluster'].unique():
    cd_col = f'Cluster {cluster} CD'
    if cd_col in df_diff_expr.columns:
        # top up genes
        up_genes = df_diff_expr.loc[df_diff_expr[cd_col].sort_values(ascending=False).iloc[:200].index, 'Symbol'].values
        # top down genes
        dn_genes = df_diff_expr.loc[df_diff_expr[cd_col].sort_values(ascending=True).iloc[:200].index, 'Symbol'].values
    else:
        raise Exception('Cant find col for cluster')
    all_top_genes.append(up_genes[:heatmap_top_n])
    all_top_genes.append(dn_genes[:heatmap_top_n])
    # save results
    top_genes[cluster] = (up_genes, dn_genes)
all_top_genes = [item for sublist in all_top_genes for item in sublist]  # flatten all genes to one list

Data corresponding to only the top 50 up- and down-regulated genes for each cluster is selected for visualization in a heatmap, with log-transformation and normalization proceeding as before. 

In [ ]:
df_data_norm_heatmap_f = df_data.loc[all_top_genes, :]

# compute log normalization of matrix
df_data_norm_heatmap_f = log2_normalize(df_data_norm_heatmap_f)

# convert to zscores
df_data_norm_heatmap_f = zscore_normalize(df_data_norm_heatmap_f) 

# Plot heatmap
cases = df_data_norm_heatmap_f.columns
heatmap_cluster_colors = [ cluster_colors[x] for x in df_clustered_umap.loc[cases, :]["Cluster"] ]

In [ ]:
sns.clustermap(df_data_norm_heatmap_f,xticklabels=False,col_colors = heatmap_cluster_colors); plt.show()
figure_legend("Figure 9", "Heatmap of most differentially expressed genes", "Color coding along the top edge indicates cluster designation of the corresponding sample.")

##  7. Enrichment Analysis with Enrichr

Perform enrichment analysis for each cluster by querying the [Enrichr](https://maayanlab.cloud/Enrichr/) API. The background libraries are the default libraries from Enrichr. A link is provided to download the results. 

In [ ]:
# enrichment analysis libraries

enrichr_libraries = OrderedDict([
    ('Diseases/Drugs', ['GWAS_Catalog_2019']), 
    ('Ontologies', ['GO_Biological_Process_2018']),
    ('Pathways', ['KEGG_2019_Human']),
    ('Transcription', ['ENCODE_TF_ChIP-seq_2015']),
])

In [ ]:
# Util functions
def enrichr_link_from_genes(genes, description='', enrichr_link='https://amp.pharm.mssm.edu/Enrichr'):
    ''' Functional access to Enrichr API
    '''
    time.sleep(1)
    resp = requests.post(enrichr_link + '/addList', files={
    'list': (None, '\n'.join(genes)),
    'description': (None, description),
    })
    if resp.status_code != 200:
        raise Exception('Enrichr failed with status {}: {}'.format(
          resp.status_code,
          resp.text,
        ))
    # wait a tinybit before returning link (backoff)
    time.sleep(3)
    result = resp.json()
    return dict(result, link=enrichr_link + '/enrich?dataset=' + resp.json()['shortId'])

def enrichr_get_top_results(userListId, bg, enrichr_link='https://amp.pharm.mssm.edu/Enrichr'):
    time.sleep(1)
    resp = requests.get(enrichr_link + '/enrich?userListId={}&backgroundType={}'.format(userListId, bg))
    if resp.status_code != 200:
        raise Exception('Enrichr failed with status {}: {}'.format(
          resp.status_code,
          resp.text,
        ))
    time.sleep(3)
    return pd.DataFrame(resp.json()[bg], columns=['rank', 'term', 'pvalue', 'zscore', 'combinedscore', 'overlapping_genes', 'adjusted_pvalue', '', ''])

In [ ]:
# Get Enrichr links for each cluster
enrichr_links = {}

for cluster, (up_genes, dn_genes) in top_genes.items():
    up_link, dn_link = None, None
    if up_genes.size:
        try:
            up_link = enrichr_link_from_genes(up_genes, f'cluster {cluster} up')
        except:
            print(f'Enrichr failed for cluster {cluster} up genes')
    else:
        print(f'cluster {cluster} up: empty')
    if dn_genes.size:
        try:
            dn_link = enrichr_link_from_genes(dn_genes, f'cluster {cluster} down')
        except:
            print(f'Enrichr failed for cluster {cluster} down genes')
    else:
        print(f'cluster {cluster} down: empty')
    enrichr_links[cluster] = (up_link, dn_link)

# Grab top results for each cluster
all_enrichr_results = []
for cluster, (up_link, dn_link) in enrichr_links.items():
    for link_type, link in [('up', up_link), ('down', dn_link)]:
        if link is None:
            continue
        for category, libraries in enrichr_libraries.items():
            for library in libraries:
                try:
                    results = enrichr_get_top_results(link['userListId'], library).sort_values('pvalue').iloc[:5]
                    results['link'] = link['link']
                    results['library'] = library
                    results['category'] = category
                    results['direction'] = link_type
                    results['cluster'] = cluster
                    all_enrichr_results.append(results)
                except:
                    print('{}: {} {} {} cluster {} failed, continuing'.format(link, library, category, link_type, cluster))

df_enrichr_results = pd.concat(all_enrichr_results).reset_index()

In [ ]:
# Display a dataframe with clickable enrichr links
figure_legend("Table 10","Enrichment analysis results from Enrichr", "Results are grouped by expression direction (up/down) and gene set library. Within groups, results are sorted by lowest p-value (highest rank) first.")
df_clickable = df_enrichr_results.copy()
df_clickable['link'] = df_clickable["link"].apply(make_clickable)
table_html = df_clickable.to_html(escape=False)
display(HTML(f'<div style="max-height: 250px; overflow-y: auto; margin-bottom: 25px;">{table_html}</div>'))
download_button(df_enrichr_results.to_csv(), 'Download Enrichr results', 'Enrichr results.csv')

### 7a. Barplots
Horizontal barplots are used to display the top Enrichr results for each cluster, by library and characteristic expression direction.

In [ ]:
# Make horizontal barplots to visualize top Enrichr results
clusters = df_enrichr_results["cluster"].unique()
for cluster in clusters:
    cluster_results = df_enrichr_results.loc[df_enrichr_results["cluster"] == cluster, :]
    libraries = cluster_results["library"].unique()
    num_rows = len(libraries)

    count = 1 # keep track of which subplot we're on
    fig = plt.figure(figsize=(15,5*num_rows))
    
    for library in cluster_results["library"].unique():
            library_results = cluster_results.loc[cluster_results["library"] == library, :]
            for direction in library_results["direction"].unique():
                plot_results = library_results.loc[cluster_results["direction"] == direction, :]
                plot_results = plot_results.sort_values("pvalue",ascending=False)
                labels = plot_results["term"]
                labels = [ '\n'.join(wrap(l, 20)) for l in labels ]
                values = plot_results["pvalue"]
                values = -np.log(values)
                
                # normalize values to map from 0-1 -> color, with opacity also based on normalized pvalue
                cmap = plt.get_cmap('cool')
                norm_values = [ 0.3 + (x - min(values))/(max(values) - min(values))*0.7 for x in values]
                colors = [ [*cmap(val)[:3], 0.4  + 0.2*val] for val in norm_values]
                
                # plot result
                ax = fig.add_subplot(num_rows,2,count)
                ax.barh(labels,values,color = colors)
                ax.set_title(f'{library}\n{direction} genes')
                ax.set_xlabel(' – log(pvalue)')
                count += 1
                
    cluster_heading(cluster)
    fig.tight_layout(pad=3, w_pad=2, h_pad=6)
    plt.show()
    display(HTML("<br><br>"))
    
figure_legend("Figure 11", "Enrichment results by cluster", "Bar plots indicate the negative log of the p-value for the specified term. One plot is presented per cluster, per gene-set library, per expression direction (up/down).")

### 7b. Running Sum Visualizations
While the above barplots display the top enriched terms for each cluster in each direction, individual enriched terms can also be compared to the tissue data using a random walk [GSEA running sum visualization](https://github.com/MaayanLab/react-GSEA/tree/master).

First, each of the four default background libraries from Enrichr can be queried and saved as a JSON object which maps terms to their complete genesets.

In [ ]:
libresp = {}
for lib in df_enrichr_results['library'].unique():
    resp = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=json&libraryName=' + lib)
    if resp.status_code == 200:
        libresp[lib] = resp.json()[lib]['terms']
    else: 
        print(f"Failed to access library {lib}, continuing")

For each cluster, the most enriched term for that cluster from each library can then be compared against the most up-regulated genes in the cluster. Below, GSEA plots display the overlap between the genes from each cluster and their most enriched genesets. 

In [ ]:
# iterate through each cluster
for cluster in clusters:
    cluster_heading(cluster)

    # iterate through each library for each cluster
    for lib in libresp.keys():

        # obtain the most enriched library term for the cluster in the up direction
        up_df = df_enrichr_results[df_enrichr_results.direction.isin(['up'])
                                    & df_enrichr_results.cluster.isin([cluster])
                                    & df_enrichr_results.library.isin([lib])]
        top_up_term = up_df[up_df['rank'] == 1]['term'].iloc[0]

        # store the geneset for the most enriched term
        top_up_set = libresp[lib][top_up_term].keys()

        display(HTML(f"<div style='font-size:1.25rem;'><b>Comparison of up-regulated genes in Cluster {cluster} to most enriched {lib} term</b> </div>"))
        print(f"Most enriched {lib} geneset for up-regulated genes:", top_up_term)

        # display the GSEA plot comparing the enriched genes and the top up-regulated cluster genes
        display(ReactGSEA(
            data=dataFromResult(
                input_set=top_up_set,
                ranked_entities=df_diff_expr['Cluster ' + cluster + ' CD'].sort_values(ascending=False).iloc[:ath.ceil((df_diff_expr.shape[0]/2))].index.tolist()
            )
        ))

In [ ]:
libresp[lib]